In [1]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from web3 import Web3
from src.config import ALCHEMY_API_KEY

ETHEREUM_RPC_URL = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(ETHEREUM_RPC_URL))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block) # latest block

True
AttributeDict({'hash': HexBytes('0x374ebe769c8df1bfe7e75c1cbcbb38c13527b49b178ff2c8942d8adaf91053e4'), 'parentHash': HexBytes('0x5e4b2a182796ba3edcddca191a9e92675fc63732457e038a9bdd910722412f68'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'miner': '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97', 'stateRoot': HexBytes('0x9f27752ad31d033e8dcfd7d31e309a0acb525faca762c46b9d603877fc99b4a8'), 'transactionsRoot': HexBytes('0x02232c105ac7329c4bba8974c0b539c6b82a39910f00a3294e5dc5983151ea23'), 'receiptsRoot': HexBytes('0x5964162e49482c390c59397de6a73efae2419dcda71ed0bb1430f1261e647493'), 'logsBloom': HexBytes('0x353f73fa6bbf2e77fe9f026eb89f5fb9db6bced3cc41dc21aba3df41faf770befd1dd71fabffb0ae7b975a431e9ecfeda76fec39af9feff71ff3b6f919af2196cc36ad7b53559ffee9a37b298c61fbb5587f3fb93edf6eae3c3e96adfbb4184ef7f5e9e59f778367ffd79b816827defbb4eb266234588e3ff3eb2d7fb4ff5f470ee04f6cdbfcfcbdf9edbee4b7faff6bbd1d5f57c5e3f8fdfea86fdf3cbf7efbffecdf339f37e9d

# Setup

In [2]:
from src.utils.retrieveAbi import save_abi_to_file
from src.config import MAINNET_UNISWAP_V2_ROUTER_02, MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET

# Load ABIs
save_abi_to_file(MAINNET_UNISWAP_V2_ROUTER_02) # uniswap v2 router abi
save_abi_to_file(MAINNET_SUSHISWAP_ROUTER_ADDRESS) # sushiswap router abi

ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/1_0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D_abi.json
ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/1_0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F_abi.json


# Contract Interactions

In [5]:
import time
from src.utils.retrieveAbi import load_abi

def get_amounts_out(token1_address, token2_address, amount_in, router_contract):
    """Uses w3 getAmountsOut function to return amounts out, given two tokens, router contract and amount in"""
    token1_address = Web3.to_checksum_address(token1_address)
    token2_address = Web3.to_checksum_address(token2_address)
    path = [token1_address, token2_address]
    amounts_out = router_contract.functions.getAmountsOut(amount_in, path).call()
    return w3.from_wei(amounts_out[-1], 'ether')

In [6]:
# uniswap
uniswap_router_abi = load_abi(MAINNET_UNISWAP_V2_ROUTER_02, CHAINID_MAINNET)
uniswap_router_address = Web3.to_checksum_address(MAINNET_UNISWAP_V2_ROUTER_02)
uniswap_router_contract = w3.eth.contract(address=uniswap_router_address, abi=uniswap_router_abi)
uniswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
uniswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
uniswap_amount_in = w3.to_wei(1, "ether")

# sushiswap
sushiswap_router_abi = load_abi(MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET)
sushiswap_router_address = Web3.to_checksum_address(MAINNET_SUSHISWAP_ROUTER_ADDRESS)
sushiswap_router_contract = w3.eth.contract(address=sushiswap_router_address, abi=sushiswap_router_abi)
sushiswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
sushiswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
sushiswap_amount_in = w3.to_wei(1, "ether")

try:
    prev_block=0
    while True:
        block = w3.eth.block_number
        uniswap_amount_out = get_amounts_out(uniswap_weth_address, uniswap_dai_address, uniswap_amount_in, uniswap_router_contract)
        sushiswap_amount_out = get_amounts_out(sushiswap_weth_address, sushiswap_dai_address, sushiswap_amount_in, sushiswap_router_contract)
        if block > prev_block:
            print(f"Block {block} | Uniswap: 1 ETH ≈ {uniswap_amount_out} DAI | Sushiswap: 1 ETH ≈ {sushiswap_amount_out} | Delta = {uniswap_amount_out-sushiswap_amount_out}")
            prev_block = block
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped by user.")

Block 23012168 | Uniswap: 1 ETH ≈ 3821.839405525931914452 DAI | Sushiswap: 1 ETH ≈ 3821.366829372867942771 | Delta = 0.472576153063971681
Stopped by user.


# Binance API

In [30]:
import pandas as pd

# Load the CSV
df = pd.read_csv("/home/tobias/personal-dex-trading/src/binance/order_book.csv", index_col="time")
print(df.head())  # Display the first few rows
print(df.columns)  # Check the column names


                                                          bids  \
time                                                             
10960954008  [['3789.95000000', '0.00280000'], ['3789.94000...   
10960954246  [['3789.72000000', '3.56520000'], ['3789.70000...   
10960954315  [['3789.72000000', '10.83340000'], ['3789.7100...   
10960954415  [['3789.72000000', '2.34450000'], ['3789.60000...   
10960954863  [['3789.36000000', '11.70140000'], ['3789.3100...   

                                                          asks  
time                                                            
10960954008  [['3789.96000000', '21.46500000'], ['3790.0000...  
10960954246  [['3789.73000000', '23.99730000'], ['3789.9900...  
10960954315  [['3789.73000000', '12.61110000'], ['3789.9900...  
10960954415  [['3789.73000000', '13.01570000'], ['3789.9100...  
10960954863  [['3789.37000000', '12.22220000'], ['3789.4000...  
Index(['bids', 'asks'], dtype='object')


In [31]:
import ast  # To convert strings back into lists

# Expand top bid and ask into separate columns with prices and quantities
def parse_order(order_list_str):
    try:
        # Convert the string to a list (e.g., '[["3789.95000000", "0.00280000"]]' -> [['3789.95000000', '0.00280000']])
        orders = ast.literal_eval(order_list_str)
        if len(orders) > 0:
            # Return the first price and quantity (top of the order book)
            return float(orders[0][0]), float(orders[0][1])
        else:
            return None, None
    except (ValueError, SyntaxError):
        return None, None

# Apply parsing to extract the top bid and ask
df[['bid_price', 'bid_quantity']] = df['bids'].apply(parse_order).apply(pd.Series)
df[['ask_price', 'ask_quantity']] = df['asks'].apply(parse_order).apply(pd.Series)

# Drop nested lists columns if no longer needed
df = df.drop(columns=['bids', 'asks'])

# Preview the result
print(df.head())


             bid_price  bid_quantity  ask_price  ask_quantity
time                                                         
10960954008    3789.95        0.0028    3789.96       21.4650
10960954246    3789.72        3.5652    3789.73       23.9973
10960954315    3789.72       10.8334    3789.73       12.6111
10960954415    3789.72        2.3445    3789.73       13.0157
10960954863    3789.36       11.7014    3789.37       12.2222


In [2]:
from src.binance.order_book import BinanceClient
binance_client = BinanceClient()

In [ ]:
"wss://stream.binance.com:9443/ws/ethusdc@depth10@1000ms" # 10 depth, 1 sec

In [38]:
import time
import datetime
from binance import ThreadedWebsocketManager
from src.config import BINANCE_API_KEY, BINANCE_API_SECRET

def handle_book_ticker(msg):
    ts = datetime.datetime.now()
    print(f"[{ts}] Symbol: {msg['s']}, Bid: {msg['b']}, Ask: {msg['a']}")
    # Du kannst hier auch in eine CSV oder Datenbank loggen, z. B.:
    # df = pd.DataFrame([{
    #     'timestamp': ts,
    #     'symbol': msg['s'],
    #     'bid_price': float(msg['b']),
    #     'ask_price': float(msg['a']),
    # }])
    # df.to_csv('ethusdc_quotes.csv', mode='a', header=False, index=False)

# Starte den WebSocket Manager
twm = ThreadedWebsocketManager(api_key=BINANCE_API_KEY, api_secret=BINANCE_API_SECRET)
twm.start()

# Starte den BookTicker-Stream für ETHUSDC
twm.start_book_ticker_socket(callback=handle_book_ticker, symbol="ethusdc")

# WebSocket läuft im Hintergrund → optional hier idle halten
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping stream...")
    twm.stop()

TypeError: ThreadedWebsocketManager.start_book_ticker_socket() got an unexpected keyword argument 'symbol'

In [133]:
exchange_info = binance_client.client.get_exchange_info()
symbols = exchange_info['symbols']
symbols_with_trading_status = [
    item['symbol'] for item in exchange_info['symbols'] if item['status'] == 'TRADING'
]
symbols_with_trading_status
#symbols

['ETHBTC',
 'LTCBTC',
 'BNBBTC',
 'NEOBTC',
 'QTUMETH',
 'GASBTC',
 'BNBETH',
 'BTCUSDT',
 'ETHUSDT',
 'LRCBTC',
 'LRCETH',
 'QTUMBTC',
 'ZRXBTC',
 'KNCBTC',
 'IOTABTC',
 'IOTAETH',
 'LINKBTC',
 'LINKETH',
 'XVGETH',
 'MTLBTC',
 'ETCETH',
 'ETCBTC',
 'ZECBTC',
 'ZECETH',
 'DASHBTC',
 'DASHETH',
 'REQBTC',
 'TRXBTC',
 'TRXETH',
 'POWRBTC',
 'POWRETH',
 'XRPBTC',
 'XRPETH',
 'ENJBTC',
 'STORJBTC',
 'BNBUSDT',
 'BATBTC',
 'NEOUSDT',
 'LSKBTC',
 'MANABTC',
 'MANAETH',
 'ADXBTC',
 'ADXETH',
 'ADABTC',
 'ADAETH',
 'XLMBTC',
 'XLMETH',
 'LTCETH',
 'LTCUSDT',
 'LTCBNB',
 'ICXBTC',
 'RLCBTC',
 'RLCETH',
 'PIVXBTC',
 'STEEMBTC',
 'STEEMETH',
 'ZILETH',
 'ONTBTC',
 'QTUMUSDT',
 'WANBTC',
 'SYSBTC',
 'ADAUSDT',
 'ADABNB',
 'XRPUSDT',
 'BTCTUSD',
 'ETHTUSD',
 'ZENBTC',
 'THETABTC',
 'XRPBNB',
 'TUSDUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'IOTXBTC',
 'IOTXETH',
 'DATABTC',
 'ONTUSDT',
 'TRXBNB',
 'TRXUSDT',
 'ETCUSDT',
 'ETCBNB',
 'ICXUSDT',
 'SCETH',
 'DENTETH',
 'ARDRBTC',
 'HOTETH',
 'VETBTC',
 'VETETH'

In [ ]:
import pandas as pd
import time

token_0 = 'ETH'
token_1 = 'USDC'

columns = ['timestamp', 'symbol', 'type', 'price', 'quantity']
df = pd.DataFrame(columns=columns)

try:
    while True:
        for symbol in symbols_with_trading_status:
            bids, asks = binance_client.get_order_book(symbol, limit=10)
            timestamp = pd.Timestamp.now()
            bid_data = pd.DataFrame(
                [[timestamp, symbol, 'bid', float(bid[0]), float(bid[1])] for bid in bids],
                columns=columns
            )
            ask_data = pd.DataFrame(
                [[timestamp, symbol, 'ask', float(ask[0]), float(ask[1])] for ask in asks],
                columns=columns
            )
            df = pd.concat([df, bid_data, ask_data], ignore_index=True)
        time.sleep(0.75)
except KeyboardInterrupt:
    print("Data collection stopped.")

/tmp/ipykernel_18598/1001304773.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, bid_data, ask_data], ignore_index=True)


Data collection stopped.


In [128]:
df.timestamp.unique()

<DatetimeArray>
['2025-07-28 14:11:38.179248', '2025-07-28 14:11:38.426308',
 '2025-07-28 14:11:38.679655', '2025-07-28 14:11:38.943721',
 '2025-07-28 14:11:39.191639', '2025-07-28 14:11:39.441002',
 '2025-07-28 14:11:39.688291', '2025-07-28 14:11:39.938269',
 '2025-07-28 14:11:40.188114', '2025-07-28 14:11:40.444159',
 '2025-07-28 14:11:40.691985', '2025-07-28 14:11:40.940186',
 '2025-07-28 14:11:41.201693', '2025-07-28 14:11:41.456804',
 '2025-07-28 14:11:41.706598', '2025-07-28 14:11:41.955987',
 '2025-07-28 14:11:42.203328', '2025-07-28 14:11:42.454587',
 '2025-07-28 14:11:42.703106', '2025-07-28 14:11:42.961157',
 '2025-07-28 14:11:43.209221', '2025-07-28 14:11:43.457887',
 '2025-07-28 14:11:43.707301', '2025-07-28 14:11:43.961409',
 '2025-07-28 14:11:44.215367', '2025-07-28 14:11:44.470755',
 '2025-07-28 14:11:44.720367', '2025-07-28 14:11:45.010833',
 '2025-07-28 14:11:45.272386', '2025-07-28 14:11:45.522406',
 '2025-07-28 14:11:45.783089', '2025-07-28 14:11:46.034957',
 '2025-0

In [130]:
df[df.symbol=="ETHBTC"]

,timestamp,symbol,type,price,quantity
0,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03264,2.5919
1,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03263,48.5632
2,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03262,71.6119
3,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03261,56.5913
4,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03260,74.9725
5,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03259,40.1652
6,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03258,61.7542
7,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03257,98.3121
8,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03256,51.4992
9,2025-07-28 14:11:38.179248,ETHBTC,bid,0.03255,40.2587


In [127]:
from IPython.display import display, HTML

display(HTML(df.to_html(notebook=True, max_rows=None, max_cols=None)))

,timestamp,symbol,type,price,quantity
0,2025-07-28 14:11:38.179248,ETHBTC,bid,3.264000e-02,2.591900e+00
1,2025-07-28 14:11:38.179248,ETHBTC,bid,3.263000e-02,4.856320e+01
2,2025-07-28 14:11:38.179248,ETHBTC,bid,3.262000e-02,7.161190e+01
3,2025-07-28 14:11:38.179248,ETHBTC,bid,3.261000e-02,5.659130e+01
4,2025-07-28 14:11:38.179248,ETHBTC,bid,3.260000e-02,7.497250e+01
5,2025-07-28 14:11:38.179248,ETHBTC,bid,3.259000e-02,4.016520e+01
6,2025-07-28 14:11:38.179248,ETHBTC,bid,3.258000e-02,6.175420e+01
7,2025-07-28 14:11:38.179248,ETHBTC,bid,3.257000e-02,9.831210e+01
8,2025-07-28 14:11:38.179248,ETHBTC,bid,3.256000e-02,5.149920e+01
9,2025-07-28 14:11:38.179248,ETHBTC,bid,3.255000e-02,4.025870e+01


Next, use SBE Market data for lower latency: https://developers.binance.com/docs/binance-spot-api-docs/sbe-market-data-streams

# Unichain

In [3]:
from src.uniswap.v2_client import UniswapV2Client
from src.config import UNICHAIN_WETH, UNICHAIN_USDC

uniswap_client = UniswapV2Client()

uniswap_client.get_amounts_out(UNICHAIN_WETH, UNICHAIN_USDC)

ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/130_0xEf740bf23aCaE26f6492B10de645D6B98dC8Eaf3_abi.json
ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/130_0x284F11109359a7e1306C3e447ef14D38400063FF_abi.json


TypeError: UniswapV2Client.get_amounts_out() missing 2 required positional arguments: 'amount_in' and 'decimals_in'

In [1]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from src.utils.retrieveAbi import save_abi_to_file, load_abi
from src.config import (
    UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, CHAINID_UNICHAIN,
    UNICHAIN_UNISWAP_V2_ROUTER_02, UNICHAIN_WETH, UNICHAIN_USDC,
    ALCHEMY_UNICHAIN_BASE_RPC_URL)

save_abi_to_file(UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, chain_id=CHAINID_UNICHAIN)
save_abi_to_file(UNICHAIN_UNISWAP_V2_ROUTER_02, chain_id=CHAINID_UNICHAIN)

Exception: An error occurred during save_abi_to_file: [Errno 2] No such file or directory: '/home/tobias/personal-dex-trading/notebooks/out/../abis/130_0xEf740bf23aCaE26f6492B10de645D6B98dC8Eaf3_abi.json'

In [19]:
WETH_AMOUNT_IN = 0.01
USDC_AMOUNT_IN = 38
weth_wei_amount_in = w3.to_wei(WETH_AMOUNT_IN, "ether")
usdc_amounts_in = USDC_AMOUNT_IN * 10 ** 6 # USDC has 6 decimals

router_address = Web3.to_checksum_address(UNICHAIN_UNISWAP_V2_ROUTER_02)
router_abi = load_abi(router_address, CHAINID_UNICHAIN)
router_contract = w3.eth.contract(address=router_address, abi=router_abi)

alchemy_rpc_url = f"{ALCHEMY_UNICHAIN_BASE_RPC_URL}{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(alchemy_rpc_url))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block.number) # latest block number

True
22964635


In [25]:
amounts_out = router_contract.functions.getAmountsOut(weth_wei_amount_in, [UNICHAIN_WETH, UNICHAIN_USDC]).call()
print(f"Input Amount (WETH): {WETH_AMOUNT_IN} WETH")
print(f"Output Amount (USDC): {amounts_out[-1] / (10 ** 6)} USDC")  # USDC has 6 decimals
print(f"----")
amounts_out_reverse = router_contract.functions.getAmountsOut(usdc_amounts_in, [UNICHAIN_USDC, UNICHAIN_WETH]).call()
print(f"Input Amount (USDC): {USDC_AMOUNT_IN} USDC")
print(f"Output Amount (WETH): {w3.from_wei(amounts_out_reverse[-1], "ether")} WETH ")  # USDC has 6 decimals

Input Amount (WETH): 0.01 WETH
Output Amount (USDC): 37.690478 USDC
----
Input Amount (USDC): 38 USDC
Output Amount (WETH): 0.009916395053523925 WETH 


Next, we can either do:
- arbitrage between Uniswap v2 and Binance 


- arbitrage between Uniswap v2 and v4 pools